<a href="https://colab.research.google.com/github/EduardoMoraesRitter/introducao-a-sistemas-de-recomendacao-com-python/blob/master/KNN_Gerando_recomenda%C3%A7%C3%B5es_baseado_em_um_usu%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RecSys

In [0]:
import pandas as pd

filmes = pd.read_csv('movies.csv')
notas = pd.read_csv('ratings.csv')

filmes.columns = ['filmeId', 'titulo', 'generos']
filmes = filmes.set_index('filmeId')
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']

#total de votos
filmes['total_de_votos'] = notas['filmeId'].value_counts()

#media de nota por filme
filmes['notas_medias'] = notas.groupby('filmeId').mean()['nota']

In [188]:
#filtar um usuario, pegar duas coluna, colocar com indice
notas_usuario = notas.query('usuarioId==1')
notas_usuario = notas_usuario[['filmeId', 'nota']].set_index("filmeId")
notas_usuario.head()

,nota
filmeId,
307.0,3.5
481.0,3.5
1091.0,1.5
1257.0,4.5
1449.0,4.5


#Parametros para teste

In [0]:
def notas_usuario(usuarioId):
    notas_usuario = notas.query('usuarioId==%d' % usuarioId)
    notas_usuario = notas_usuario[['filmeId', 'nota']].set_index("filmeId")
    return notas_usuario

In [0]:
def distancia_usuarios(uId1,uId2, minimo=5):
  u1N = notas_usuario(uId1)
  u2N = notas_usuario(uId2)
  diferencas = u1N.join(u2N, lsuffix="u_esq", rsuffix="u_dir").dropna()
  if(len(diferencas) < minimo):
    return None
  distancia_usu1_usu2 = distancia_vetor(diferencas['notau_esq'], diferencas['notau_dir'])
  return [uId1, uId2, distancia_usu1_usu2]

In [0]:
def distancia_todos(voce, numero_usuario_analizar = None):
  todos_usuarios = notas['usuarioId'].unique()
  if numero_usuario_analizar:
     todos_usuarios = todos_usuarios[:numero_usuario_analizar]
  distancias = [distancia_usuarios(voce,usuario_id) for usuario_id in todos_usuarios]
  distancias = list(filter(None, distancias))
  distancias = pd.DataFrame(distancias, columns = ["voce", "outro", "distancia"])
  return distancias

In [192]:
def mais_proxima(voce_id, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id)
  return distancias
mais_proxima(1, numero_usuario_analizar=5)

,voce,distancia
outro,,
4,1,3.041381


#pegar o usuario mais similar, pegar os filmes que ele assistiu e vc nao 

In [193]:
voce = 1
notas_voce = notas_usuario(voce)
filmes_voce = notas_voce.index
similares = mais_proxima(voce, numero_usuario_analizar=5)
#pega a linha iloc, pega o name
similar = similares.iloc[0].name
notas_similar = notas_usuario(similar)
#remover os filmes q vc assistiu da lista, que vc viu q ele nao viu
notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
notas_similar = notas_similar.sort_values("nota", ascending=False)
notas_similar.head()

,nota
filmeId,
2193.0,5.0
27773.0,5.0
349.0,5.0
8132.0,5.0
8360.0,5.0


In [0]:
def sugere_para(voce, numero_usuario_analizar=None):
  notas_voce = notas_usuario(voce)
  filmes_voce = notas_voce.index
  similares = mais_proxima(voce, numero_usuario_analizar=numero_usuario_analizar)
  similar = similares.iloc[0].name
  notas_similar = notas_usuario(similar)
  notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
  recomendacoes = notas_similar.sort_values("nota", ascending=False)
  return recomendacoes.join(filmes)

In [0]:
sugere_para(1).head()

#sugerir baseado em varios usuarios

In [0]:
def mais_proxima(voce_id, n_mais_proximos=10, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id)
  return distancias.head(n_mais_proximos)

In [0]:
mais_proxima(1, 3, 200)

In [0]:
voce = 1
n_mais_proximos = 10
numero_usuario_analizar = 200

notas_voce = notas_usuario(voce)
filmes_voce = notas_voce.index

similares = mais_proxima(voce, n_mais_proximos=n_mais_proximos, numero_usuario_analizar=numero_usuario_analizar)
usuarios_similares = similares.index
#pega a nota de todos esse usuario
#notas.query('usuarios in [4,6]') muito ruim
#nao tem problema do index esta repetido
notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]

#fazer uma media dentre as notas, retorna um serie
notas_similares.groupby("filmeId").mean()["nota"]

#retona com [] transforma para dataframe
recomendacoes = notas_similares.groupby("filmeId").mean()[["nota"]]
recomendacoes = recomendacoes.sort_values("nota", ascending=False)

#junta com os filmes
recomendacoes.join(filmes).head()

#antes
# similar = similares.iloc[0].name
# notas_similar = notas_usuario(similar)
# notas_similar = notas_similar.drop(filmes_voce, errors='ignore')
# notas_similar = notas_similar.sort_values("nota", ascending=False)
# notas_similar.head()

In [0]:
def knn(voce, k_mais_proximos = 10, numero_usuario_analizar = None):
  notas_voce = notas_usuario(voce)
  filmes_voce = notas_voce.index

  similares = mais_proxima(voce, n_mais_proximos=k_mais_proximos, numero_usuario_analizar=numero_usuario_analizar)
  usuarios_similares = similares.index
  notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_similares.groupby("filmeId").mean()[["nota"]]
  recomendacoes = recomendacoes.sort_values("nota", ascending=False)
  return recomendacoes.join(filmes)

In [0]:
knn(1, 10, 50).head()

In [0]:
knn(1).head()

In [0]:
knn(1).tail()

#Notas de filmes com mais de 50 notas


In [0]:
filmes_mais_50_votos = filmes.query("total_de_votos >= 50")
filmes_mais_50_votos.index
#colocar as nota para index ser o filme, localiza so os mais de 50 votos
notas = notas.set_index("filmeId").loc[filmes_mais_50_votos.index]

notas.head()

In [0]:
#precisa voltar o index original, para qunado tentar acessa a coluna filmeID nao da
notas = notas.reset_index()
notas.head()

In [0]:
knn(1).head()

In [0]:
#cuidado com o drop pq ele pode nao estar la usar o errors='ignore'
def mais_proxima(voce_id, n_mais_proximos=10, numero_usuario_analizar = None):
  distancias = distancia_todos(voce_id, numero_usuario_analizar=numero_usuario_analizar)
  distancias = distancias.sort_values("distancia").set_index("outro").drop(voce_id, errors='ignore')
  return distancias.head(n_mais_proximos)

In [0]:
#tirar os filmes que tem pouco votos dentro da pessoas que sao semelhantes a mim
def knn(voce_id, k_mais_proximos = 10, numero_usuario_analizar = None):
  notas_voce = notas_usuario(voce_id)
  filmes_voce = notas_voce.index

  similares = mais_proxima(voce_id, n_mais_proximos=k_mais_proximos, numero_usuario_analizar=numero_usuario_analizar)
  usuarios_similares = similares.index
  notas_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_similares.groupby("filmeId").mean()[["nota"]]
  
  aparicoes = notas_similares.groupby("filmeId").count()[['nota']]
  filtro_minimo = k_mais_proximos / 2
  recomendacoes = recomendacoes.join(aparicoes, lsuffix='_media_usuarios', rsuffix='_qtd_votos_usuarios')
  recomendacoes = recomendacoes.query("nota_qtd_votos_usuarios >= %.2f" % filtro_minimo)
  recomendacoes = recomendacoes.sort_values("nota_media_usuarios", ascending=False)

  recomendacoes = recomendacoes.drop(filmes_voce, errors='ignore')

  return recomendacoes.join(filmes)

In [0]:
knn(1).head()